In [1]:
!pip install python-docx trafilatura markdown-it-py mdit_plain pypdf python-pptx openpyxl nltk

# SuperComponents

Supercomponents in general behave like any other component. They have init params, optional from_dict() and to_dict() methods as usual. The init params typically determine how the internal pipeline is constructed (e.g. which components are used).

In [2]:
from haystack_experimental.super_components.converters import AutoFileConverter

The AutoFileConverter converts multiple file types and splits them into smaller chunks.

In [ ]:
file_converter = AutoFileConverter(
    split_by="sentence",
    split_overlap=0,
    split_length=1,
    respect_sentence_boundary=False
)

It handles CSV, DOCX, HTML, JSON, Markdown, TXT, PDF, PPTX, and XLSX.

In [8]:
example_files = [
    "example_files/react_paper.pdf",
    "example_files/sample_docx.docx",
    "example_files/sample_pptx.pptx",
    "example_files/sample.md",
    "example_files/sample_1.csv",
]

result = file_converter.run(sources=example_files)

previous_name = None
for document in result["documents"]:
    if document.meta["file_path"] != previous_name:
        doc = f"""
First lines from {document.meta["file_path"]}:
{document.content}
-----------
"""  
        print(doc)
        previous_name = document.meta["file_path"]

Converting markdown files to Documents: 100%|██████████| 1/1 [00:00<00:00, 324.49it/s]



First lines from sample_1.csv:
Name,Age
John Doe,27
Jane Smith,37
Mike Johnson,47

-----------


First lines from sample_docx.docx:
Sample Docx File

The US has "passed the peak" on new coronavirus cases, President Donald Trump said and predicted that some states would reopen this month.

-----------


First lines from sample.md:
type: intro
date: 1.1.2023

pip install farm-haystack


What to build with Haystack

Ask questions in natural language and find granular answers in your own documents.

-----------


First lines from react_paper.pdf:
REAC T: S YNERGIZING REASONING AND ACTING IN
LANGUAGE MODELS
Shunyu Yao∗*,1, Jeffrey Zhao2, Dian Yu2, Nan Du2, Izhak Shafran2, Karthik Narasimhan1, Yuan Cao2
1Department of Computer Science, Princeton University
2Google Research, Brain team
1{shunyuy,karthikn}@princeton.edu
2{jeffreyzhao,dianyu,dunan,izhak,yuancao}@google.com
ABSTRACT
While large language models (LLMs) have demonstrated impressive performance
across tasks in language understandin

In [4]:
# We can serialize it to a dict like any other Haystack component.
file_converter.to_dict()

{'type': 'haystack_experimental.super_components.converters.file_converter.AutoFileConverter',
 'init_parameters': {'split_by': 'word',
  'split_length': 250,
  'split_overlap': 30,
  'split_threshold': 0,
  'splitting_function': None,
  'respect_sentence_boundary': True,
  'language': 'en',
  'use_split_rules': True,
  'extend_abbreviations': True,
  'encoding': 'utf-8',
  'json_content_key': 'content'}}

## Expanding SuperComponents
What makes SuperComponents special is the ability to expand it by calling their `to_super_component_dict()` method. This converts the component to a generic `SuperComponent` that contains the pipeline constructed by the SuperComponent. From there on the pipeline can be changed in any way.

In [5]:
file_converter.to_super_component_dict()

{'type': 'haystack_experimental.core.super_component.super_component.SuperComponent',
 'init_parameters': {'pipeline': {'metadata': {},
   'max_runs_per_component': 100,
   'components': {'router': {'type': 'haystack.components.routers.file_type_router.FileTypeRouter',
     'init_parameters': {'mime_types': [<ConverterMimeType.CSV: 'text/csv'>,
       <ConverterMimeType.DOCX: 'application/vnd.openxmlformats-officedocument.wordprocessingml.document'>,
       <ConverterMimeType.HTML: 'text/html'>,
       <ConverterMimeType.JSON: 'application/json'>,
       <ConverterMimeType.MD: 'text/markdown'>,
       <ConverterMimeType.TEXT: 'text/plain'>,
       <ConverterMimeType.PDF: 'application/pdf'>,
       <ConverterMimeType.PPTX: 'application/vnd.openxmlformats-officedocument.presentationml.presentation'>,
       <ConverterMimeType.XLSX: 'application/vnd.openxmlformats-officedocument.spreadsheetml.sheet'>],
      'additional_mimetypes': None}},
    'csv': {'type': 'haystack.components.converte

In [10]:
from haystack_experimental.core.super_component import SuperComponent

super_file_converter = SuperComponent.from_dict(file_converter.to_super_component_dict())

In [11]:
result = super_file_converter.run(sources=example_files)

previous_name = None
for document in result["documents"]:
    if document.meta["file_path"] != previous_name:
        doc = f"""
First lines from {document.meta["file_path"]}:
{document.content}
-----------
"""  
        print(doc)
        previous_name = document.meta["file_path"]

No abbreviations file found for en. Using default abbreviations.
Converting markdown files to Documents: 100%|██████████| 1/1 [00:00<00:00, 380.50it/s]



First lines from sample_1.csv:
Name,Age
John Doe,27
Jane Smith,37
Mike Johnson,47

-----------


First lines from sample_docx.docx:
Sample Docx File

The US has "passed the peak" on new coronavirus cases, President Donald Trump said and predicted that some states would reopen this month.

-----------


First lines from sample.md:
type: intro
date: 1.1.2023

pip install farm-haystack


What to build with Haystack

Ask questions in natural language and find granular answers in your own documents.

-----------


First lines from react_paper.pdf:
REAC T: S YNERGIZING REASONING AND ACTING IN
LANGUAGE MODELS
Shunyu Yao∗*,1, Jeffrey Zhao2, Dian Yu2, Nan Du2, Izhak Shafran2, Karthik Narasimhan1, Yuan Cao2
1Department of Computer Science, Princeton University
2Google Research, Brain team
1{shunyuy,karthikn}@princeton.edu
2{jeffreyzhao,dianyu,dunan,izhak,yuancao}@google.com
ABSTRACT
While large language models (LLMs) have demonstrated impressive performance
across tasks in language understandin